## Proyecto en Equipo: Producto de Datos sobre la bd Chicago Food Inspections.
### Maestría en Ciencia de Datos, ITAM
**Data Product Architecture (aka Métodos de Gran Escala)**  
**Prof. M. Sc. Liliana Millán Núñez**
_______
> **Equipo: Luigi007. Github:**

| Github User  | Alumno			   |
|:------------:|:-----------------:|
| @Carlosrlpzi | LS, CARLOS ROMAN  |
| @urieluard   | RD, URIEL ABRAHAM |
| @            | EMPTY             |
| @ZarCorvus   | ZC, JOSE LUIS R.  |

_______

#### Base de datos
[Chicago Food Inspections](https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5)  
Pregunta analítica: **¿El establecimiento pasará o no la inspección?**

_______
**Checkpoint 1: Starting point**
#### EDA, Data Profiling

* 4 visualizaciones de los datos
* 1 mapa indicando las inspecciones que pasaron y las que no (histórico)
_______

#### Conocimiento a priori de la información:
* **_Title:_** Food Inspections

* **_Descripción:_**  
  * La información se obtiene de las inspecciones de restaurants y otros establecimientos de comida en Chicago desde 01/01/2010 hasta la actualidad.
  * Las inspecciones se realizan por personal del Chicago Department of Public Health’s Food Protection Program siguiendo un procedimiento estandarizado.
  * Se debe precisar que en **01/07/2018 se realizaron modificaciones** a los procedimientos de inspección que afectan a los datos. Estructuralmente el dataset no será afectado. No obstante, la columna **"Violations"**, si bien aún contiene el número de violación, descripción y comentarios delimitando violaciones independientes con el "pipe character", las violaciones actuales se modificaron substancialmente. Debemos ser cuidadosos al analizar los datos en los posibles cambios de tendencia en el largo plazo. [FOOD INSPECTION VIOLATIONS CHANGES](http://bit.ly/2yWd2JB).
  * En adición, encontramos la modificación a los siguientes términos (**validar en la información**) [2019 Chicago Food Code Major Changes ](https://www.cityofchicago.org/city/en/depts/cdph/provdrs/healthy_restaurants/svcs/food-protection-services.html.):
     * Potentially Hazardous Foods (PHF)" has been changed to “Time/Temperature Control for Safety Foods (TCS Foods)"
     * "Critical Violation" has been changed to “Priority (P) Violation"
     * "Serious Violation" has been changed to "Priority Foundation (PF)Violation"
     * "Minor Violation" has been changed to "Core (C) Violation"
     * "Corrected During Inspection (CDI)" has been changed to "Corrected on Site (COS)"
 
* **_METADATA ¿qué sabemos de las variables? y posible tratamiento:_**  
Se realizará limpieza sobre los nombres de columnas del dataset: lowercase, underscore para espacios y sustitución de símbolos. En particular, observaremos lo siguiente en cada variable:

  * **Inspection ID**: N/I. Analizar si este ID nos resulta funcional, posible variable a eliminar
  * **DBA Name:** Nombre legal del establecimiento. Analizar si es una variable funcional comparando con "AKA Name"
  * **AKA Name:** Nombre público como se conoce al establecimiento. Facilita diferenciar cadenas comerciales de negocios locales. Revisar vs DBA para determinar cual varible se debe conservar. 
  * **License #:** Número **único** asignado al establecimiento con fines de licenciamiento.
  * **Facility Type:** Esta variable debe estar fuertemente correlacionada con que un negocio pase o no la inspección. Analizar:  
    <<bakery, banquet hall, candy store, caterer, coffee shop, day care center (for ages less than 2), day care center (for ages 2 – 6), day care center (combo, for ages less than 2 and 2 – 6 combined), gas station, Golden Diner, grocery store, hospital, long term care center(nursing home), liquor store, mobile food dispenser, restaurant, paleteria, school,shelter, tavern, social club, wholesaler, or Wrigley Field Rooftop.>>  
  * **Risk:** Esta variable de estar fuertemente correlacionada con "Facility Type", revisar. Cada establecimiento se categoriza de acuerdo al riesgo de afectar la salud pública. 1 el más alto riesgo y 3 el menor. **La frecuencia de las inspecciones está ligada a su nivel de riesgo.**
  * **Address, City, State, Zip:** 4 variables que componen la dirección completa donde se localizan las instalaciones. Es altamente probable que para nuestro análisis únicamente la variable ZIP sea relevante.  
  * **Inspection Date:** Fecha de inspección. Es probable que un establecimiento específico tenga múltiples inspecciones, las cuales serán diferenciadas por la fecha. Variable muy importante para el análisis temporal.
  * **Inspection Type:**  
      canvass, the most common type of inspection performed at a frequency relative to the risk of the establishment;  
      consultation, when the inspection is done at the request of the owner prior to the opening of the establishment;  
      complaint, when the inspection is done in response to a complaint against the establishment;  
      license, when the inspection is done as a requirement for the establishment to receive its license to operate;  
      suspect food poisoning, when the inspection is done in response to one or more persons claiming to have gotten ill as a result of eating at the establishment;  
      task-force inspection, when an inspection of a bar or tavern is done.  
      **_Re-inspections can occur for most types of these inspections and are indicated as such._**
  * **Results:** **TARGET VARIABLE**. Pass, pass with conditions, fail, out of business or not located; 'pass' implica que no se tienen violaciones críticas o severas (códigos de violación 1-14 y 15-29 respectivamente). 'pass with conditions', se encontraton violaciones críticas o severas, pero fueron corregidas durante la inspección. 'fail' implica que se tienen violaciones críticas o severas y que no se corrigieron durante la inspección. 
  * **Violations:** Variable relevante para nuestro análisis. Un establecimiento puede recibir más de una de las 45 distintas violaciones (código de violación 1 al 44 y 70).
  * **Latitude, Longitude, Location:** N/I. Variables sumamente importantes para construir mapas. Se considera que "Location" es una variable redundante; no obstante, previo a depurarla, se debe analizar que la composición de "Latitude" y "Longitude" sea congruente con "Location" y, en caso necesario y factible, completar info faltante.


* **_Disclaimer:_**  
El dataset puede contener reportes **duplicados** por lo que deben considerarse las precauciones adecuadas al visualizar y analizar esta información. Los resultdos de la inspección (pass, pass with conditions or fail) así como las violaciones ("violations") se basan en los hallazgos reportados por los inspectores al momento de la inspección y podrían no reflejar los hallazgos en otros momentos. 

* **_Frecuencia:_** Esta base de datos se actualiza con información de nuevas inspecciones cada viernes.

In [3]:
# loading libraries required

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib


In [6]:
# load database

fi_data = pd.read_csv('Food_Inspections.csv')


In [5]:
fi_data.nunique()

Inspection ID      215067
DBA Name            28748
AKA Name            27355
License #           39103
Facility Type         500
Risk                    4
Address             18522
City                   70
State                   4
Zip                   112
Inspection Date      2796
Inspection Type       110
Results                 7
Violations         156694
Latitude            17246
Longitude           17246
Location            17246
dtype: int64